Fonte: https://basedosdados.org/dataset/f06f3cdc-b539-409b-b311-1ff8878fb8d9?table=a3696dc2-4dd1-4f7e-9769-6aa16a1556b8

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

import locale
try:
    locale.setlocale(locale.LC_ALL, 'Portuguese_Brazil')
except:
    pass

In [ ]:
df = pd.read_parquet('../data/queimadas_data-2015-2025.parquet')
df.head()

Como os dados de 2025 estão incompletos, vamos removê-los.

In [ ]:
df = df[df['ano'] != 2025]
df.shape

Em seguida, removemos dados que não são informativos ou que são únicos, como o ID do município e o satélite que fez a captura.

In [ ]:
df.drop(columns=['id_municipio', 'satelite'], inplace=True)
df.head()

In [ ]:
df.info()

## Visualizações iniciais
TODO: adicionar/melhorar mais visualizações

In [ ]:
records_per_year = df['ano'].value_counts().sort_index()

df_plot = records_per_year.reset_index()
df_plot.columns = ['ano', 'registros']
df_plot['registros_formatado'] = df_plot['registros'].apply(
    lambda x: f"{x:_.0f}".replace('_', '.')
)

fig = px.bar(
    df_plot,
    x='ano',
    y='registros',
    labels={'ano': 'Ano', 'registros': 'Número de registros'},
    title='Número de Registros de Queimadas por Ano',
    custom_data=['registros_formatado']
)

fig.update_layout(
    title={
        'text': 'Número de Registros de Queimadas por Ano',
        'x': 0.5,
        'xanchor': 'center'
    }
)

fig.update_traces(
    hovertemplate='<b>Ano:</b> %{x}<br>' +
                  '<b>Número de registros:</b> %{customdata[0]}' +
                  '<extra></extra>'
)

fig.show()

Como há uma quantidade massiva de dados, para o plot abaixo, vamos agrupar os dados em uma grade de ~30km.

In [ ]:
df['lat_bin'] = (df['latitude'] // 0.3) * 0.3
df['lon_bin'] = (df['longitude'] // 0.3) * 0.3

df_agg = df.groupby(['ano', 'lat_bin', 'lon_bin']).size().reset_index(name='count')

print(f"Redução: {len(df):,} → {len(df_agg):,} pontos")

fig = px.density_map(
    df_agg,
    lat='lat_bin',
    lon='lon_bin',
    z='count',
    animation_frame='ano',
    radius=15,
    zoom=3,
    center=dict(lat=-14.2, lon=-51.9),
    map_style='carto-positron',
    color_continuous_scale='YlOrRd',
    title='Distribuição Geográfica das Queimadas no Brasil por Ano',
    height=700,
    range_color=[0, None]
)

fig.update_layout(
    title={'x': 0.5, 'xanchor': 'center'},
    sliders=[{
        'currentvalue': {
            'prefix': 'Ano: ',
            'font': {'size': 20}
        }
    }]
)

fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000
fig.layout.updatemenus[0].buttons[0].args[1]["transition"]["duration"] = 500

fig.show()

### Análise univariada
TODO

### Análise multivariada
TODO

## Verificação de valores nulos, indefinidos e duplicados

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

Como temos valores indefinidos para a nossa target variable (potencia_radiativa_fogo), vamos removê-los.

In [ ]:
df.dropna(subset=['potencia_radiativa_fogo'], inplace=True)
df.shape

In [ ]:
df.isna().sum()

## Visualização dos dados faltantes

**Padrão inicial identificado**: os dados faltantes estão concentrados nas variáveis climáticas. **Hipótese**: pode ter ocorrido algum problema na coleta/integração da fonte de dados.

In [ ]:
df_na = df[df.isna().any(axis=1)]
df_na.shape

### Distribuição temporal
Descoberta: os dados faltantes estão concentrados nos anos de 2023 e 2024.

In [ ]:
na_per_year = df_na.groupby('ano').size().reset_index(name='count_na')
total_per_year = df.groupby('ano').size().reset_index(name='count_total')

temporal = na_per_year.merge(total_per_year, on='ano')
temporal['perc_na'] = (temporal['count_na'] / temporal['count_total'] * 100).round(2)

fig = px.bar(
    temporal,
    x='ano',
    y='perc_na',
    text='perc_na',
    labels={'perc_na': '% de NAs', 'ano': 'Ano'},
    title='Porcentagem de Dados Climáticos Faltantes por Ano',
    height=500
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(title={'x': 0.5, 'xanchor': 'center'})
fig.show()

### Distribuição espacial
Descoberta: os dados faltantes pertencem majoritariamente às regiões Norte, Nordeste e Centro-Oeste do Brasil.

In [ ]:
df_agg = df_na.groupby(['lat_bin', 'lon_bin', 'bioma']).size().reset_index(name='count')

fig = px.density_map(
    df_agg,
    lat='lat_bin',
    lon='lon_bin',
    z='count',
    radius=10,
    zoom=3.5,
    center=dict(lat=-14.2, lon=-51.9),
    map_style='carto-positron',
    color_continuous_scale='YlOrRd',
    title="Distribuição Espacial de Registros com Dados Climáticos Faltantes",
    height=700
)
fig.update_layout(title={'x': 0.5, 'xanchor': 'center'})
fig.show()

### Distribuição por bioma
Descoberta: o bioma com maior quantidade de dados faltantes é a Amazônia.

In [ ]:
# Prepara dados
na_bioma = df_na['bioma'].value_counts().reset_index()
na_bioma.columns = ['bioma', 'count_na']

total_bioma = df['bioma'].value_counts().reset_index()
total_bioma.columns = ['bioma', 'count_total']

bioma_comp = na_bioma.merge(total_bioma, on='bioma')
bioma_comp['perc_na'] = (bioma_comp['count_na'] / bioma_comp['count_total'] * 100).round(2)

# Formata contagem para hover
bioma_comp['count_na_fmt'] = bioma_comp['count_na'].apply(
    lambda x: f"{x:_.0f}".replace('_', '.')
)
bioma_comp['count_total_fmt'] = bioma_comp['count_total'].apply(
    lambda x: f"{x:_.0f}".replace('_', '.')
)

# Scatter plot com tamanho proporcional ao total de registros
fig = px.scatter(
    bioma_comp,
    x='perc_na',
    y='count_na',
    size='count_total',
    color='bioma',
    text='bioma',
    size_max=60,
    custom_data=['count_na_fmt', 'count_total_fmt'],
    labels={
        'perc_na': 'Porcentagem de Dados Faltantes (%)',
        'count_na': 'Quantidade Absoluta de NAs',
        'count_total': 'Total de Registros'
    },
    title='Dados Climáticos Faltantes por Bioma',
    height=700,
    color_discrete_sequence=px.colors.qualitative.Set2
)

# Ajusta posição dos labels
fig.update_traces(
    textposition='top center',
    hovertemplate='<b>%{text}</b><br>' +
                  'Porcentagem: %{x:.1f}%<br>' +
                  'Quantidade absoluta: %{customdata[0]}<br>' +
                  'Total de registros: %{customdata[1]}' +
                  '<extra></extra>',
)

fig.update_layout(
    title={'x': 0.5, 'xanchor': 'center'},
    xaxis=dict(title_font=dict(size=14)),
    yaxis=dict(title_font=dict(size=14)),
    showlegend=False,
    font=dict(size=12)
)

fig.show()

### Correlação com a intensidade do fogo
Descoberta: não há diferença significativa na distribuição da potência radiativa entre os registros com e sem dados climáticos.

In [ ]:
n_na = len(df_na)

df_comparacao = pd.concat([
    df_na.assign(grupo='Com NAs'),
    df[df['dias_sem_chuva'].notna()].sample(n=n_na, random_state=42).assign(grupo='Sem NAs')
])

fig = px.box(
    df_comparacao,
    x='grupo',
    y='potencia_radiativa_fogo',
    title='Distribuição de Potência Radiativa: Registros com vs. sem NAs climáticos'
)
fig.update_layout(title={'x': 0.5, 'xanchor': 'center'})
fig.show()

## Tratamento dos dados restantes faltantes
Como os dados ausentes estão concentradados espaço-temporalmente, adicionar uma flag para indicar a ausência dos dados climáticos pode ser uma boa estratégia, uma vez que possibilita ao modelo aprender padrões relacionados à falta desses dados. Em seguida, faremos a imputação dos valores faltantes utilizando a mediana das respectivas colunas.

In [ ]:
from sklearn.impute import SimpleImputer, MissingIndicator

clima_cols = ['dias_sem_chuva', 'precipitacao', 'risco_fogo']

indicator = MissingIndicator(missing_values=np.nan, features='all')
flags = indicator.fit_transform(df[clima_cols])

df['clima_faltante'] = flags.any(axis=1).astype(int)

imputer = SimpleImputer(strategy='median')
df[clima_cols] = imputer.fit_transform(df[clima_cols])

print(f"Registros com clima faltante (clima_faltante=1): {df['clima_faltante'].sum():,}")
print(f"NaNs restantes: {df[clima_cols].isna().sum().sum()}")

In [ ]:
df[df['clima_faltante'] == 1].head()

## Exportação dos dados tratados

### Regressão

In [ ]:
df.to_parquet('data/queimadas-regressao.parquet', index=False)

### Classificação
Para a tarefa de classificação, vamos categorizar a variável target (potencia_radiativa_fogo) em três classes: baixa, média e alta intensidade de fogo, utilizando o método de quantis para garantir uma distribuição equilibrada entre as classes. Depois, removeremos a coluna original de potência radiativa do fogo para evitar data leakage.

> O FRP sofre influência de ângulo de observação, condições de relevo, condições atmosféricas e **não existe uma classificação pré-definida e nem uma escala que permita alguma inferência do tipo da combustão**. Por exemplo, o sensor no satélite irá indicar para um píxel o mesmo FRP tanto de uma pequena área queimando com alta temperatura, como de uma área maior, com combustão de menor temperatura. É necessário trabalhar com os dados explorando seus valores para cada região e período, a partir de valores de referências de alvos conhecidos no solo. Assim, o uso do FRP para casos pontuais implica em medidas imprecisas, porém em uma escala regional os resultados podem indicar valores médios aceitáveis da massa de vegetação queimada, e neste contexto o Programa Queimadas do INPE calcula e divulga os valores de FRP (INPE, 2020).

In [ ]:
df['label_intensidade'] = pd.qcut(df['potencia_radiativa_fogo'], q=3, labels=['baixa', 'media', 'alta'])
df.drop(columns=['potencia_radiativa_fogo'], inplace=True)
df.head()

In [ ]:
df['label_intensidade'].value_counts()

In [ ]:
df.to_parquet('data/queimadas-classificacao.parquet', index=False)